## Limpieza de Datos: Compras Públicas Municipalidades

Este notebook documenta el proceso de limpieza y estandarización de datos de compras públicas municipales en Chile, obtenidos de la plataforma Mercado Público. El objetivo es preparar los datasets para un EDA posterior.

Para correr este notebook es necesario tener un una carpeta llamada Municipalidad con los datos en formato csv, en el mismo directorio. el formato es:

"Limpieza/Municipalidades/CompraAgil.csv" 


Se trabajará con 4 datasest: 

*  Compra Ágil
* Convenio Marco
* Licitación
* Trato Directo


!! Por ahora no trabajare con Convenio Marco, más adelante verémos si es de utilidad



In [24]:
import pandas as pd
import os

In [25]:
def clean_dataset(df, data_name):
    """
    Función para limpiar tipos de datos en datasets de compras públicas
    """

    # Fechas
    if 'FechaEnvioOC' in df.columns:
            df['FechaEnvioOC'] = pd.to_datetime(df['FechaEnvioOC'], dayfirst=True, errors='coerce')
    
    # Montos
    for col in ['MontoNetoOC_CLP', 'MontoNetoItemCLP', 'CantidadItem']:
        if col in df.columns:
            #arreglar formatos
            df[col] = df[col].astype(str).str.replace('.', '', regex=False)
            df[col] = df[col].str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # para trabajar con enteros
            df[col] = df[col].round().astype('Int64')
    
    # Limpiar texto
    text_columns = ['Proveedor', 'Institucion', 'NombreOC', 'NombreItem']
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    
    return df

In [26]:
'''
Aqui voy a seleccionar algunas columnas de interés
'''

columns_to_keep = [
    "codigoOC",
    "NroLicitacion",
    "FechaEnvioOC",
    "EstadoOC",
    "NombreOC",
     
    # Institución compradora
    "Institucion",
    "UnidadCompra",
    "RegionUnidadCompra",
    "Financiamiento",

    # Proveedor
    "Proveedor",
    "ProveedorRUT",
    "TamanoProveedor",
    "RegionProveedor",

    # Montos y compra
    "MontoNetoOC_CLP",
    "MontoNetoItemCLP",
    "ProcedenciaOC",

    # Producto / Servicio
    "NombreItem",
    "CantidadItem",
    "RubroN1",
    "RubroN2",
    "RubroN3",
    "ONUProducto"
]
df = pd.read_csv("Municipalidades/Licitacion.csv", sep=";", encoding="latin1", usecols=columns_to_keep)

In [27]:
'''
Limpieza básica de eliminación de nulos y duplicados
'''

df = df.drop_duplicates()

df = clean_dataset(df, "Licitacion")

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234742 entries, 0 to 250821
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   codigoOC            234742 non-null  object        
 1   NroLicitacion       234742 non-null  object        
 2   FechaEnvioOC        234742 non-null  datetime64[ns]
 3   NombreOC            234742 non-null  object        
 4   EstadoOC            234742 non-null  object        
 5   ProcedenciaOC       234742 non-null  object        
 6   MontoNetoOC_CLP     234707 non-null  Int64         
 7   Financiamiento      210968 non-null  object        
 8   UnidadCompra        234742 non-null  object        
 9   RegionUnidadCompra  233886 non-null  object        
 10  Institucion         234742 non-null  object        
 11  Proveedor           234742 non-null  object        
 12  ProveedorRUT        234742 non-null  object        
 13  TamanoProveedor     234742 non-nul

In [29]:
# para los siguientes datasts, hay que sacar "NroLicitacion"
columns_to_keep_2 = columns_to_keep.copy()  
columns_to_keep_2.pop(1)

'NroLicitacion'

In [30]:

df_td = pd.read_csv("Municipalidades/TratoDirecto.csv", sep=";", encoding="latin1", usecols=columns_to_keep_2)

In [32]:
'''
Limpieza básica de eliminación de nulos y duplicados
'''

df_td = df_td.drop_duplicates()
df_td = clean_dataset(df_td, "TratoDirecto")

/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_5825/1245742677.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FechaEnvioOC'] = pd.to_datetime(df['FechaEnvioOC'], dayfirst=True, errors='coerce')
/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_5825/1245742677.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).str.replace('.', '', regex=False)
/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_5825/1245742677.py:15: SettingWithCopyWarning:

In [ ]:
df_td.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41941 entries, 0 to 43857
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   codigoOC            41941 non-null  object        
 1   FechaEnvioOC        41941 non-null  datetime64[ns]
 2   NombreOC            41941 non-null  object        
 3   EstadoOC            41941 non-null  object        
 4   ProcedenciaOC       41941 non-null  object        
 5   MontoTotalOC        41941 non-null  Int64         
 6   MontoNetoOC_CLP     41913 non-null  Int64         
 7   UnidadCompra        41941 non-null  object        
 8   RegionUnidadCompra  41941 non-null  object        
 9   Institucion         41941 non-null  object        
 10  Sector              41941 non-null  object        
 11  Proveedor           41941 non-null  object        
 12  ProveedorRUT        41941 non-null  object        
 13  TamanoProveedor     41941 non-null  object        


In [ ]:
df_ca = pd.read_csv("Municipalidades/CompraAgil.csv", sep=";", encoding="latin1", usecols=columns_to_keep_2)

/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_4415/2197230537.py:1: DtypeWarning: Columns (13,35,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ca = pd.read_csv("Municipalidades/CompraAgil.csv", sep=";", encoding="latin1", usecols=columns_to_keep_2)


In [ ]:
df_ca = df_ca.dropna(subset=["Proveedor", "MontoTotalOC"])
df_ca = df_ca.drop_duplicates()
df_ca = clean_dataset(df_ca, "CompraAgil")

In [ ]:
df_ca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 426455 entries, 0 to 2492920
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   codigoOC            426455 non-null  object        
 1   FechaEnvioOC        426455 non-null  datetime64[ns]
 2   NombreOC            426455 non-null  object        
 3   EstadoOC            426455 non-null  object        
 4   ProcedenciaOC       426455 non-null  object        
 5   MontoTotalOC        426455 non-null  Int64         
 6   MontoNetoOC_CLP     426455 non-null  Int64         
 7   UnidadCompra        426455 non-null  object        
 8   RegionUnidadCompra  426455 non-null  object        
 9   Institucion         426455 non-null  object        
 10  Sector              426455 non-null  object        
 11  Proveedor           426455 non-null  object        
 12  ProveedorRUT        426455 non-null  object        
 13  TamanoProveedor     426455 non-nu

In [ ]:
# Concatenar datasets
df_combined = pd.concat([df, df_td, df_ca], ignore_index=True)
print(f"Dataset combinado: {df_combined.shape}")

# crear carpeta si no existe
output_dir = "../datos_limpios"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  
# Guardar archivo
output_file = "../datos_limpios/municipalidades_combinado.csv"
df_combined.to_csv(output_file, index=False, sep=";", encoding="utf-8")

Dataset combinado: (703144, 23)
